Replicate [Dynamic Return Dependencies Across Industries: A Machine Learning Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3120110&download=yes) by David Rapach, Jack Strauss, Jun Tu and Guofu Zhou.

1) Use Keras NNs instead of linear regression

2) Add additional variables, 3 and 12-month MA, interest rate change, yield curve, Mkt-RF. The hope is with  cross-validation and regularization we can do that without overfitting.


In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import pandas_datareader.data as datareader
import time 
import datetime
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import tensorflow as tf
tf.set_random_seed(1764)

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Input
from keras.models import Model

from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l1
from keras.models import Sequential
from keras.models import load_model

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


Using TensorFlow backend.


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

    
# add rates data from FRED
start_date = datetime.datetime(1926, 9, 1)
end_date = datetime.datetime(2017, 12, 1)
TB3MS = datareader.DataReader("TB3MS", "fred", start_date, end_date)
TB3MS['yyyymm'] = TB3MS.index.strftime('%Y%m')
TB3MS['yyyymm'] = [int(datestr) for datestr in TB3MS['yyyymm']]
TB3MS=TB3MS.set_index(['yyyymm'])
data['3month']=TB3MS['TB3MS']

GS10 =  datareader.DataReader("GS10", "fred", start_date, end_date)
GS10['yyyymm'] = GS10.index.strftime('%Y%m')
GS10['yyyymm'] = [int(datestr) for datestr in GS10['yyyymm']]
GS10=GS10.set_index(['yyyymm'])
data['10year']=GS10['GS10']

data['curve'] = data['10year'] - data['3month']
data['10year'] = data['10year'].diff() # first difference 10-year yield
data['3month'] = data['3month'].diff() # first difference 3-month
data['month'] = (data.index  % 100)/12.0 # for possible seasonality
data['Mkt-RF'] = rfdata['Mkt-RF']

for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
    data[ind+".12m"] = pd.rolling_mean(data[ind],12)

for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)
print(list(data.columns))
data[['3month', '10year', 'curve', 'month', 'Mkt-RF',]]

Loading data...
(697, 133)
['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other', '3month', '10year', 'curve', 'month', 'Mkt-RF', 'Food.3m', 'Beer.3m', 'Smoke.3m', 'Games.3m', 'Books.3m', 'Hshld.3m', 'Clths.3m', 'Hlth.3m', 'Chems.3m', 'Txtls.3m', 'Cnstr.3m', 'Steel.3m', 'FabPr.3m', 'ElcEq.3m', 'Autos.3m', 'Carry.3m', 'Mines.3m', 'Coal.3m', 'Oil.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'BusEq.3m', 'Paper.3m', 'Trans.3m', 'Whlsl.3m', 'Rtail.3m', 'Meals.3m', 'Fin.3m', 'Other.3m', '3month.3m', '10year.3m', 'curve.3m', 'Mkt-RF.3m', 'Food.12m', 'Beer.12m', 'Smoke.12m', 'Games.12m', 'Books.12m', 'Hshld.12m', 'Clths.12m', 'Hlth.12m', 'Chems.12m', 'Txtls.12m', 'Cnstr.12m', 'Steel.12m', 'FabPr.12m', 'ElcEq.12m', 'Autos.12m', 'Carry.12m', 'Mines.12m', 'Coal.12m', 'Oil.12m', 'Util.12m', 'Telcm.1

,3month,10year,curve,month,Mkt-RF
yyyymm,,,,,
195912,0.34,0.16,0.20,1.000000,2.45
196001,-0.14,0.03,0.37,0.083333,-6.98
196002,-0.39,-0.23,0.53,0.166667,1.17
196003,-0.65,-0.24,0.94,0.250000,-1.63
196004,-0.08,0.03,1.05,0.333333,-1.71
196005,0.06,0.07,1.06,0.416667,3.12
196006,-0.83,-0.20,1.69,0.500000,2.08
196007,-0.16,-0.25,1.60,0.583333,-2.37
196008,0.00,-0.10,1.50,0.666667,3.01


In [3]:
#data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,...,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000
mean,0.688666,0.727030,0.985079,0.732095,0.532253,0.564333,0.690387,0.665825,0.552367,0.687145,...,0.515968,0.729928,0.622970,0.534806,0.601090,0.631076,0.698235,0.728766,0.637547,0.396628
std,4.308660,5.058992,6.032324,7.128170,5.780362,4.728000,6.355251,4.897557,5.482363,6.970961,...,4.607931,6.486956,6.698787,5.021876,5.707154,5.571040,5.334178,6.065564,5.381389,5.771655
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.630000,-2.080000,-2.740000,-3.390000,-2.600000,-2.030000,-2.800000,-2.230000,-2.750000,-3.170000,...,-2.110000,-3.050000,-3.220000,-2.400000,-2.780000,-2.560000,-2.380000,-2.840000,-2.400000,-2.930000
50%,0.740000,0.750000,1.270000,0.940000,0.510000,0.750000,0.700000,0.760000,0.720000,0.640000,...,0.590000,1.010000,0.670000,0.710000,0.900000,0.940000,0.540000,1.080000,0.870000,0.540000
75%,3.070000,3.690000,4.660000,5.260000,3.640000,3.540000,4.310000,3.550000,3.760000,4.480000,...,3.360000,4.260000,4.630000,3.460000,4.040000,3.880000,3.980000,4.300000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# Run LASSO, then OLS on selected variables

# We get better R-squareds across the board in-sample, unsurprisingly

X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(696, 103)

In [6]:
def subset_selection(X, Y, model_aic, verbose=False):
    
    global responses
    global response_reverse_dict
    global predictors
    global predictor_reverse_dict
    
    coef_dict = {}
    for response_index, response in enumerate(responses):
        y = Y[:,response_reverse_dict[response]]
        
        model_aic.fit(X, y)

        coef_dict[response] = [predstr for i, predstr in enumerate(predictors) if model_aic.coef_[i] !=0]
        #y_response = model_aic.responseict(X)
        # print ("In-sample LASSO R-squared: %.6f" % r2_score(y, y_response))
        if verbose:
            print("LASSO variables selected for %s: " % response)
            print(coef_dict[response])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response + ", using all")
                print("---")
            coef_dict[response] = predictors            
        # fit OLS vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        if verbose:
            print("Running OLS for " + response + " against " + str(coef_dict[response]))
            # col nums of selected responses
            predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
            model_ols = LinearRegression()
            model_ols.fit(X[:, predcols], y)
            y_pred = model_ols.predict(X[:, predcols])
            print ("In-sample OLS R-squared: %.2f%%" % (100 * r2_score(y, y_pred)))
            print("---")
            
    return coef_dict

coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'), verbose=True)


LASSO variables selected for Food.lead: 
['10year', 'Mines.12m']
Running OLS for Food.lead against ['10year', 'Mines.12m']
In-sample OLS R-squared: 2.26%
---
LASSO variables selected for Beer.lead: 
['Food', 'Clths', 'Coal', '3month', '10year', 'Beer.3m', 'Hlth.3m', 'Util.3m', 'Mines.12m', 'Coal.12m', 'Servs.12m', '3month.12m']
Running OLS for Beer.lead against ['Food', 'Clths', 'Coal', '3month', '10year', 'Beer.3m', 'Hlth.3m', 'Util.3m', 'Mines.12m', 'Coal.12m', 'Servs.12m', '3month.12m']
In-sample OLS R-squared: 6.64%
---
LASSO variables selected for Smoke.lead: 
['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', '3month', 'month', 'Food.3m', 'Beer.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Servs.3m', 'Paper.3m', 'Other.3m', 'Food.12m', 'Smoke.12m', 'Games.12m', 'Hshld.12m', 'Hlth.12m', 'FabPr.12m', 'Paper.12m', '10year.12m']
Running OLS for Smoke.lead against ['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', '3month', 'month', 'Food.3m', 'Beer.3m', 'Chems.3m', 'Elc

LASSO variables selected for Meals.lead: 
['Books', 'Clths', 'Coal', 'Oil', 'Servs', 'Meals', '3month', 'Util.3m', 'Paper.3m', 'Paper.12m', '3month.12m', 'curve.12m']
Running OLS for Meals.lead against ['Books', 'Clths', 'Coal', 'Oil', 'Servs', 'Meals', '3month', 'Util.3m', 'Paper.3m', 'Paper.12m', '3month.12m', 'curve.12m']
In-sample OLS R-squared: 9.86%
---
LASSO variables selected for Fin.lead: 
['Books', 'Clths', 'Coal', 'Oil', 'Util', 'Fin', 'Other', '3month', '10year', 'Beer.3m', 'Coal.3m', 'Paper.3m', 'Rtail.3m', 'Books.12m', 'Chems.12m', 'FabPr.12m', 'Mines.12m', 'Coal.12m', 'Servs.12m', 'BusEq.12m', '10year.12m', 'curve.12m']
Running OLS for Fin.lead against ['Books', 'Clths', 'Coal', 'Oil', 'Util', 'Fin', 'Other', '3month', '10year', 'Beer.3m', 'Coal.3m', 'Paper.3m', 'Rtail.3m', 'Books.12m', 'Chems.12m', 'FabPr.12m', 'Mines.12m', 'Coal.12m', 'Servs.12m', 'BusEq.12m', '10year.12m', 'curve.12m']
In-sample OLS R-squared: 9.82%
---
LASSO variables selected for Other.lead: 
['Smok

In [7]:
def predict_with_subsets(X, Y, create_model, coef_dict, verbose=False):
    """similar output to subset_selection, but pass a model function and a dictionary, compute avg R-squared"""
    global responses
    global response_reverse_dict
    
    model = create_model()
    
    scores = []
    for response in responses:
        y = Y[:,response_reverse_dict[response]]

#        print("LASSO variables selected for %s: " % pred)
#        print(coef_dict[pred])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response)
 #           print("---")
            continue
        # fit model vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        #print("Running model for " + pred + " against " + str(coef_dict[pred]))
        # col nums of selected predictors
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        model.fit(X[:, predcols], y)
        y_pred = model.predict(X[:, predcols])
        score = r2_score(y, y_pred)
        scores.append(score)
        if verbose:
            print ("In-sample R-squared: %.4f%% for %s against %s" % (score*100, response, str(coef_dict[response])))
#        print("---")
    
    if verbose:
        print("Mean R-squared: %.4f%%" % (100 * np.mean(np.array(scores))))
    return np.mean(np.array(scores))
    

predict_with_subsets(X, Y, LinearRegression, coef_dict, verbose=True)


In-sample R-squared: 2.2604% for Food.lead against ['10year', 'Mines.12m']
In-sample R-squared: 6.6423% for Beer.lead against ['Food', 'Clths', 'Coal', '3month', '10year', 'Beer.3m', 'Hlth.3m', 'Util.3m', 'Mines.12m', 'Coal.12m', 'Servs.12m', '3month.12m']
In-sample R-squared: 14.2499% for Smoke.lead against ['Txtls', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', '3month', 'month', 'Food.3m', 'Beer.3m', 'Chems.3m', 'ElcEq.3m', 'Mines.3m', 'Servs.3m', 'Paper.3m', 'Other.3m', 'Food.12m', 'Smoke.12m', 'Games.12m', 'Hshld.12m', 'Hlth.12m', 'FabPr.12m', 'Paper.12m', '10year.12m']
In-sample R-squared: 4.1425% for Games.lead against ['Books', 'Fin']
In-sample R-squared: 15.0344% for Books.lead against ['Smoke', 'Games', 'Clths', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Meals', 'Fin', '3month', '10year', 'month', 'Chems.3m', 'Steel.3m', 'ElcEq.3m', 'Mines.3m', 'Coal.3m', 'Util.3m', 'Paper.3m', '10year.3m', 'Beer.12m', 'Smoke.12m', 'Books.12m', 'Chems.12m', 'Steel.12m', 'FabPr.12m

0.08330520917634218

In [8]:
coef_dict_all = {}
for response in responses:
    coef_dict_all[response] = predictors
predict_with_subsets(X, Y, LinearRegression, coef_dict_all, verbose=True)


In-sample R-squared: 16.6947% for Food.lead against ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other', '3month', '10year', 'curve', 'month', 'Mkt-RF', 'Food.3m', 'Beer.3m', 'Smoke.3m', 'Games.3m', 'Books.3m', 'Hshld.3m', 'Clths.3m', 'Hlth.3m', 'Chems.3m', 'Txtls.3m', 'Cnstr.3m', 'Steel.3m', 'FabPr.3m', 'ElcEq.3m', 'Autos.3m', 'Carry.3m', 'Mines.3m', 'Coal.3m', 'Oil.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'BusEq.3m', 'Paper.3m', 'Trans.3m', 'Whlsl.3m', 'Rtail.3m', 'Meals.3m', 'Fin.3m', 'Other.3m', '3month.3m', '10year.3m', 'curve.3m', 'Mkt-RF.3m', 'Food.12m', 'Beer.12m', 'Smoke.12m', 'Games.12m', 'Books.12m', 'Hshld.12m', 'Clths.12m', 'Hlth.12m', 'Chems.12m', 'Txtls.12m', 'Cnstr.12m', 'Steel.12m', 'FabPr.12m', 'ElcEq.12m', 'Autos.12m', 'Carry.12m', 'Mines.12m', 'Coal.12m', 'Oil.1

In-sample R-squared: 21.5174% for Paper.lead against ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other', '3month', '10year', 'curve', 'month', 'Mkt-RF', 'Food.3m', 'Beer.3m', 'Smoke.3m', 'Games.3m', 'Books.3m', 'Hshld.3m', 'Clths.3m', 'Hlth.3m', 'Chems.3m', 'Txtls.3m', 'Cnstr.3m', 'Steel.3m', 'FabPr.3m', 'ElcEq.3m', 'Autos.3m', 'Carry.3m', 'Mines.3m', 'Coal.3m', 'Oil.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'BusEq.3m', 'Paper.3m', 'Trans.3m', 'Whlsl.3m', 'Rtail.3m', 'Meals.3m', 'Fin.3m', 'Other.3m', '3month.3m', '10year.3m', 'curve.3m', 'Mkt-RF.3m', 'Food.12m', 'Beer.12m', 'Smoke.12m', 'Games.12m', 'Books.12m', 'Hshld.12m', 'Clths.12m', 'Hlth.12m', 'Chems.12m', 'Txtls.12m', 'Cnstr.12m', 'Steel.12m', 'FabPr.12m', 'ElcEq.12m', 'Autos.12m', 'Carry.12m', 'Mines.12m', 'Coal.12m', 'Oil.

0.19170032993169955

In [30]:
def fit_predict(X, Y, model, coef_dict="all", npredict=1):
    """for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final row
    if npredict=12, fit using n-12 rows, return prediction using X for final 12 rows
    coef_dict = None -> perform subset selection at each timestep using subset_selection()
    coef_dict = 'all' -> use all predictors"""
    
    global responses
    global response_reverse_dict
    
    # keep last row to predict against
    X_predict = X[-npredict:]
    nrows, ncols = X.shape
    X_predict = X_predict.reshape(npredict,ncols)
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]

    # if no coef_dict select predictors into coef_dict
    if coef_dict is None:
        coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))
    # if coef_dict == "all" use all predictors for each response        
    elif coef_dict == 'all':
        coef_dict = {}
        for response in responses:
            coef_dict[response]=predictors

    predictions = []
    for response in responses:
        if not coef_dict[response]:
            predictions.append([np.nan]*npredict)
            continue
        # column indexes to fit against each other
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        responsecol = response_reverse_dict[response]
        #print("fitting on" + str(X_fit[:, predcols].shape))
        model.fit(X_fit[:, predcols], Y_fit[:,responsecol])
        #print("predicting on" + str(X_predict[:, predcols].shape))
        y_pred = model.predict(X_predict[:,predcols])        
        predictions.append(y_pred)
        
    return np.array(predictions).transpose()

X = data.values[:,:npredictors]
Y = data.values[:, -nresponses:]
print(X[684])
print(Y[684])
predictions = fit_predict(X[:685], Y[:685], LinearRegression(), coef_dict, 1)
print(predictions)

[ 4.43        3.          5.39       -3.36        1.98        1.43
 -0.44        0.82        0.32       -1.27        0.16       -2.14
 -0.15       -0.4         4.16        1.4        -1.8        -9.68
  2.17        3.61        4.65       -0.19        2.07        1.86
  0.84        2.34       -0.98        0.58        3.8         2.57
  0.06        0.35        1.98        1.          1.82       -0.10333333
 -1.47        1.62        2.03333333 -0.05       -1.90333333 -1.46666667
 -1.75333333  1.97333333 -1.28666667  2.          5.24333333  2.73
  1.06        2.58666667  4.13333333  3.52333333  0.80333333  2.92333333
  0.04333333  2.68333333 -0.25        0.74        0.19333333  4.31666667
  1.15       -0.52666667  2.36333333  6.01333333  3.44333333  0.07333333
  0.28666667  1.7         1.55333333  0.59583333  0.49083333  1.31416667
  1.43416667  0.43        0.4925     -0.775      -0.30416667  1.34666667
 -0.4375      1.4625      3.66583333  2.84166667  1.36916667  0.79083333
  1.50833333  

In [10]:
# first iteration will train up to including 196911
# will use 196912 to predict 197001
# 19701 will be first month of performance to use
# train on first 121 months up to 196912 (0:120), put first prediction in P[121] (122nd row)
# first month of performance will be 197002
FIRST_TRAIN_MONTHS = 121
FIRST_PREDICT_MONTH = FIRST_TRAIN_MONTHS # This is stupid but keeps my head straight

print(X[FIRST_TRAIN_MONTHS])
print(data.iloc[FIRST_TRAIN_MONTHS][:30])

[-3.34000000e+00 -1.95000000e+00 -7.59000000e+00 -7.76000000e+00
 -1.20500000e+01 -7.50000000e+00 -5.69000000e+00 -7.71000000e+00
 -7.37000000e+00 -5.26000000e+00 -9.84000000e+00 -6.31000000e+00
 -7.15000000e+00 -6.89000000e+00 -9.35000000e+00 -1.24900000e+01
 -2.34000000e+00 -7.70000000e-01 -1.21600000e+01 -4.83000000e+00
 -3.16000000e+00 -1.11700000e+01 -9.73000000e+00 -8.89000000e+00
 -8.17000000e+00 -8.28000000e+00 -6.31000000e+00 -1.31200000e+01
 -9.78000000e+00 -6.20000000e+00  5.00000000e-02  1.40000000e-01
 -8.00000000e-02  8.33333333e-02 -8.10000000e+00 -2.28666667e+00
 -2.18000000e+00 -3.36000000e+00 -7.00333333e+00 -6.82000000e+00
 -3.37666667e+00 -5.32666667e+00 -1.41000000e+00 -5.58666667e+00
 -5.43333333e+00 -6.02666667e+00 -4.45000000e+00 -4.68333333e+00
 -4.67666667e+00 -5.93666667e+00 -9.37000000e+00 -2.48000000e+00
  2.38666667e+00 -6.98666667e+00 -3.96666667e+00 -2.88333333e+00
 -4.91666667e+00 -4.43000000e+00 -4.87666667e+00 -8.43000000e+00
 -6.69666667e+00 -4.78333

In [11]:
# make predictions for all months in backtest period
# initial train_months = 120 -> train first model on 120 rows
# first prediction will be in P[120] (121st row)
# step = 6 -> predict following 6 rows, then step forward 6 months at a time
# initialize global predictions matrix P

def run_backtest(X, Y, create_model, coef_dict=None, initial_train_months=0, 
                 minmaxscale=False, standardscale=False, step=1, verbose=False):
    global P
    P = np.zeros_like(Y)

    count = 0
    nrows, ncols = Y.shape
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")
        
    # create list of steps
    month_indexes = list(range(initial_train_months+step, nrows, step))
    # force an iteration for last (nrows % step) rows
    if month_indexes[-1] < nrows: 
        month_indexes.append(nrows)
    if verbose:
        print("Initial train rows = %d" % initial_train_months)
        print ("Steps: " + str(month_indexes))

    model = create_model()
    
    for month_index in month_indexes:
        # if month_index =121 and step =1
        # train on 0:119 (120 rows)
        # fit on 120 (121st row)
        # if month_index = 1000 and step = 10
        # fit on 0:989, predict 990:999
        if verbose:
            print("Training on first %d rows (%d:%d), putting prediction in rows %s" % (month_index-step, 0, month_index-step-1, str(range(month_index-step,month_index))))
        predictions = fit_predict(Xscale[:month_index], 
                                  Yscale[:month_index], 
                                  model,
                                  coef_dict,
                                  step)
        #990
        first_pred_row = month_index-step
        for row_index in range(step):
            P[first_pred_row+row_index] = predictions[row_index]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, (X.shape[0]-initial_train_months+1)))
            sys.stdout.flush()
    print("")

    msetemp = (P[initial_train_months:]-Yscale[initial_train_months:])**2
    #remove nans
    msetemp = msetemp[~np.isnan(msetemp)]
    mse = np.mean(msetemp)
    print("MSE across all predictions: %.4f" % mse)
    print("Variance: %.4f" % (np.mean(Yscale[initial_train_months:]**2)))
    print("R-squared: %.4f" % (1- mse/(np.mean(Yscale[initial_train_months:]**2))))
     


In [12]:
def gen_returns(Y, P, first_pred_month, start_date='01/01/1970', freq='M', verbose=False):
    #TODO: more general version
    #take an indicator (P), a function to generate portfolio based on indicator, universe returns (Y), return portfolio returns

    global R
    R = np.zeros(P.shape[0])
    nrows, ncols = P.shape
    numstocks = 6 # top quintile (and bottom)

    indcount = [0 for response in responses]
    longcount = [0 for response in responses]
    shortcount = [0 for response in responses]
        
    for month_index in range(first_pred_month, nrows):
        # get indexes of sorted smallest to largest
        # leftmost 6
        # ignore nan
        short_sort_array = [999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(short_sort_array)
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_sort_array = [-999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(long_sort_array)
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        return_month = month_index + 1
        if verbose:
            print("Longs for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in long_indexes])))
            print("Shorts for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in short_indexes])))
            
        if return_month < nrows: # last row has a prediction for following month but no following month
            R[return_month] = np.mean(X[return_month, long_indexes])/2 - np.mean(X[return_month, short_indexes])/2
            # count occurrences of each industry
            for i in short_indexes:
                indcount[i]+=1
                shortcount[i]+=1
            for i in long_indexes:
                indcount[i]+=1
                longcount[i]+=1

    for response in responses:
        i = response_reverse_dict[response]
        print("%s: long %d times, short %d times, total %d times" % (response, longcount[i], shortcount[i], indcount[i]))
        
    results = R[first_pred_month:]

    index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [13]:
# run linear regression for baseline
run_backtest(X, Y, LinearRegression, coef_dict, initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)

# this is even better than the paper (Sharpe 0.97)
# clearly there is impact from snooping, but we knew that
# (paper presumably uses theoretically snoop-free LASSO threshold)
# add a bunch of variables, select good ones using whole period, use the good ones in backtest
# more variables you add, better the result, more snooping effect, LASSO info leaks back into earlier performance

# OOS MSE of 39.5535 is worse, suggests overfitting
# But performance is better, suggests no overfitting. 
# Relationship between MSE/R-squared and performance seems tenuous, which is a little surprising.


................................................................................
09:46:28 Still training 80 of 577
................................................................................
09:46:30 Still training 160 of 577
................................................................................
09:46:31 Still training 240 of 577
................................................................................
09:46:33 Still training 320 of 577
................................................................................
09:46:34 Still training 400 of 577
................................................................................
09:46:36 Still training 480 of 577
................................................................................
09:46:38 Still training 560 of 577
................
MSE across all predictions: 39.6376
Variance: 39.5535
R-squared: -0.0021
Food.lead: long 77 times, short 67 times, total 144 times
Beer.lead: long 121 times, short 88 times

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0888345
yearly_vol,0.0971392
yearly_sharpe,0.967051
max_drawdown,-0.060131
sortino,0.841122


In [14]:
# double check results_post_LASSO
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results_post_LASSO = R[FIRST_PREDICT_MONTH:]
print(len(results_post_LASSO))
#print(results_post_LASSO)
print(np.mean(results_post_LASSO))
print(np.std(results_post_LASSO) * np.sqrt(12))
print(np.prod(1 + results_post_LASSO / 100))
print(np.prod(1 + results_post_LASSO / 100) ** (12.0/results_post_LASSO.shape[0]))-1

576
0.7228370949074073
5.485195287590178
59.02450150561251
0.08867063021911337


In [15]:
# run performance chart
perf_post_LASSO = 100 * np.cumprod(1 + results_post_LASSO / 100)

def mychart(args, names=None):
    x_coords = np.linspace(1970, 2016, args[0].shape[0])
    
    plotdata = []
    for i in range(len(args)):
        tracelabel = "Trace %d" % i
        if names:
                tracelabel=names[i]
        plotdata.append(Scatter(x=x_coords,
                                y=args[i].reshape(-1),
                                mode = 'line',
                                name=tracelabel))    

    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    
mychart([perf_post_LASSO],["Post-LASSO"])

In [16]:
# pass coef_dict as None
# fit_predict will do subset selection at each timestep using data it trains on
run_backtest(X, Y, LinearRegression, coef_dict=None, initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH)

# Returns not as good as with fewer variables (unsurprisingly)

................................................................................
09:47:42 Still training 80 of 577
................................................................................
09:48:35 Still training 160 of 577
................................................................................
09:49:25 Still training 240 of 577
................................................................................
09:50:11 Still training 320 of 577
................................................................................
09:50:56 Still training 400 of 577
................................................................................
09:51:41 Still training 480 of 577
................................................................................
09:52:26 Still training 560 of 577
................
MSE across all predictions: 45.1396
Variance: 39.5535
R-squared: -0.1412
Food.lead: long 75 times, short 72 times, total 147 times
Beer.lead: long 110 times, short 121 time

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0327433
yearly_vol,0.0517068
yearly_sharpe,0.639522
max_drawdown,-0.138067
sortino,0.32085


In [17]:
results_LASSO_each_timestep = R[FIRST_PREDICT_MONTH:]
perf_LASSO_each_timestep = 100 * np.cumprod(1 + results_LASSO_each_timestep / 100)
mychart([perf_LASSO_each_timestep])

In [18]:
# pass coef_dict as 'all'
# fit_predict will use all predictors (no subset selection)
run_backtest(X, Y, LinearRegression, coef_dict='all', initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH)

................................................................................
09:52:40 Still training 80 of 577
................................................................................
09:52:45 Still training 160 of 577
................................................................................
09:52:50 Still training 240 of 577
................................................................................
09:52:55 Still training 320 of 577
................................................................................
09:53:00 Still training 400 of 577
................................................................................
09:53:06 Still training 480 of 577
................................................................................
09:53:11 Still training 560 of 577
................
MSE across all predictions: 74.4120
Variance: 39.5535
R-squared: -0.8813
Food.lead: long 109 times, short 60 times, total 169 times
Beer.lead: long 114 times, short 115 tim

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0236823
yearly_vol,0.0638839
yearly_sharpe,0.400922
max_drawdown,-0.209353
sortino,0.237134


In [19]:
results_OLS = R[FIRST_PREDICT_MONTH:]
perf_OLS = 100 * np.cumprod(1 + results_OLS / 100)
mychart([perf_OLS])

In [20]:
mychart([perf_post_LASSO, perf_LASSO_each_timestep, perf_OLS],["Post-LASSO", "LASSO each timestep", "OLS"])

In [21]:
def walkforward_xval (X, Y, create_model, coef_dict="all", n_splits=5):

    start = time.time()

    # generate k-folds
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    model = create_model()
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        if coef_dict is None:
            print("Performing LASSO subset selection on training set")
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'), verbose=False)
        elif coef_dict == 'all':
            # if coef_dict == "all" use all predictors for each response
            coef_dict = {}
            for response in responses:
                coef_dict[response]=predictors
                
        mse_list = []
        
        for response in responses:
            predcols = [predictor_reverse_dict[indstr] for indstr in coef_dict[response]]
            if len(predcols) == 0:
                continue
            responsecol = response_reverse_dict[response]
            
            fit = model.fit(X_fit[:,predcols], Y_fit[:,responsecol])
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = fit.predict(X_xval[:,predcols])
            mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred))
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()             
        # mean mse over industry ys for this fold
        xval_score = np.mean(np.array(mse_list))            

        # choose model with lowest xval loss
        print ("\n%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    print ("Last Xval loss %f" % (xval_score))
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [23]:
# get baseline using linear regression
walkforward_xval (X, Y, LinearRegression, coef_dict=coef_dict)


01:19:12 Generate splits [140, 280, 419, 558, 697]
01:19:12 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
01:19:12 Xval MSE 31.315901
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
01:19:12 Xval MSE 69.570862
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
01:19:12 Xval MSE 63.481511
Last Xval loss 63.481511
Avg Xval loss 54.789425
--------------------------------------------------------------------------------


(54.78942458318909,
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))

In [22]:
# closure to wrap model creation function - return a function to create model
def create_model(n_hidden_layers, layer_size, reg_penalty):
    def create_func():
        return MLPRegressor(hidden_layer_sizes=tuple([layer_size]*n_hidden_layers),
                            alpha=reg_penalty,
                            activation='tanh',
                            max_iter=10000, 
                            tol=1e-10,
                            solver='lbfgs')
    return create_func

In [24]:
walkforward_xval (X, Y, create_model(3,2,1.0), coef_dict=coef_dict)

01:19:12 Generate splits [140, 280, 419, 558, 697]
01:19:12 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
01:19:39 Xval MSE 35.344126
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
01:20:12 Xval MSE 71.362061
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
01:20:43 Xval MSE 65.345100
Last Xval loss 65.345100
Avg Xval loss 57.350429
--------------------------------------------------------------------------------


(57.35042904355612,
 MLPRegressor(activation='tanh', alpha=1.0, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(2, 2, 2), learning_rate='constant',
        learning_rate_init=0.001, max_iter=10000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='lbfgs', tol=1e-10, validation_fraction=0.1,
        verbose=False, warm_start=False))

In [25]:
def run_experiment(X, Y, 
                   n_hidden_layers, 
                   layer_size, 
                   reg_penalty,
                   coef_dict="all", 
                   minmaxscale=False, 
                   standardscale=False):
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")

    return walkforward_xval(Xscale, Yscale, 
                            create_model(n_hidden_layers, layer_size, reg_penalty), 
                            coef_dict=coef_dict)


In [26]:
# try many combos

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4, 8]
reg_penalties = [0.0, 0.001, 0.01, 0.1, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))
experiments = {}
        
for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))    
    score, model = run_experiment (X, Y, n_hidden_layers, layer_size, reg_penalty)
    experiments[key] = score


01:20:43 Running 60 experiments
01:20:43 Running experiment 1 of 60
01:20:43 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.000000
01:20:43 Generate splits [140, 280, 419, 558, 697]
01:20:43 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
01:20:47 Xval MSE 33.841702
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
01:20:52 Xval MSE 70.716246
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
01:20:55 Xval MSE 65.850293
Last Xval loss 65.850293
Avg Xval loss 56.802747
--------------------------------------------------------------------------------
01:20:55 Running experiment 2 of 60
01:20:55 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.001000
01:20:55 Generate splits [140, 280, 419, 558, 697]
01:20:55 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
01

..............................
01:33:29 Xval MSE 55.254436
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
01:34:31 Xval MSE 97.617895
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
01:35:34 Xval MSE 83.401774
Last Xval loss 83.401774
Avg Xval loss 78.758035
--------------------------------------------------------------------------------
01:35:34 Running experiment 14 of 60
01:35:34 n_hidden_layers = 1, hidden_layer_size = 4, reg_penalty = 0.100000
01:35:34 Generate splits [140, 280, 419, 558, 697]
01:35:34 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
01:36:36 Xval MSE 95.642854
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
01:37:47 Xval MSE 118.064865
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
01:39:00 Xval MSE 90.089261
Last Xval loss

..............................
02:06:10 Xval MSE 38.549288
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
02:06:30 Xval MSE 72.898778
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
02:06:46 Xval MSE 67.255292
Last Xval loss 67.255292
Avg Xval loss 59.567786
--------------------------------------------------------------------------------
02:06:46 Running experiment 26 of 60
02:06:46 n_hidden_layers = 2, hidden_layer_size = 2, reg_penalty = 0.000000
02:06:46 Generate splits [140, 280, 419, 558, 697]
02:06:46 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
...................
02:08:45 Xval MSE 72.630741
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
02:09:21 Xval MSE 70.831766
Last Xval loss 70.831766
Avg Xval loss 60.395153
--------------------------------------------------------------------------------
02:09:21 Running 

..............................
02:58:26 Xval MSE 121.706014
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
03:00:57 Xval MSE 109.066105
Last Xval loss 109.066105
Avg Xval loss 102.319952
--------------------------------------------------------------------------------
03:00:57 Running experiment 39 of 60
03:00:57 n_hidden_layers = 2, hidden_layer_size = 8, reg_penalty = 0.100000
03:00:57 Generate splits [140, 280, 419, 558, 697]
03:00:57 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
03:02:57 Xval MSE 72.663710
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
03:05:16 Xval MSE 118.056121
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
03:07:53 Xval MSE 109.609946
Last Xval loss 109.609946
Avg Xval loss 100.109926
-------------------------------------------------------------------------------

..............................
03:32:35 Xval MSE 87.864638
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
03:34:29 Xval MSE 74.007866
Last Xval loss 74.007866
Avg Xval loss 73.662179
--------------------------------------------------------------------------------
03:34:29 Running experiment 51 of 60
03:34:29 n_hidden_layers = 3, hidden_layer_size = 4, reg_penalty = 0.000000
03:34:29 Generate splits [140, 280, 419, 558, 697]
03:34:29 Starting training
Training indexes 0 to 279
Cross-validating indexes 280 to 418
..............................
03:35:57 Xval MSE 51.264903
Training indexes 0 to 418
Cross-validating indexes 419 to 557
..............................
03:37:51 Xval MSE 86.534377
Training indexes 0 to 557
Cross-validating indexes 558 to 696
..............................
03:40:05 Xval MSE 89.627341
Last Xval loss 89.627341
Avg Xval loss 75.808874
--------------------------------------------------------------------------------
03:40:

In [27]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "loss"])
lossframe.sort_values(['loss'])

,n_hidden_layers,layer_size,reg_penalty,loss
38,3,1,0.000,54.640088
31,3,1,0.001,55.085651
43,3,1,0.010,55.532363
56,2,1,0.010,55.604385
49,2,1,0.000,55.815012
50,2,1,0.001,55.962620
0,3,1,0.100,56.107480
55,1,1,0.001,56.327276
47,1,1,0.000,56.802747
15,2,1,0.100,56.810712


In [28]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


,loss
n_hidden_layers,
1,80.864425
2,76.093601
3,75.953888


In [29]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())


,loss
layer_size,
1,56.919798
2,66.651784
4,81.577582
8,105.400056


In [30]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())


,loss
reg_penalty,
0.000,73.142936
0.001,73.397307
0.010,73.441489
0.100,79.833124
1.000,88.371668


In [31]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[y_labels], columns=[x_labels], values=['loss'])
#    print(pivot)
    # specify labels as strings, to force plotly to use a discrete axis
#    print(pivot.columns.levels[1]).values
#    print(lossframe[x_labels].dtype)
    
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
#    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" layers", y_suffix=" units")



In [32]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix=" layers", y_suffix=" rp")


In [ ]:
plot_matrix(lossframe, "layer_size", "reg_penalty", x_suffix=" units", y_suffix=" rp")


In [24]:
# these results are not very good, ~same as LinearRegression, but try best one
# 1-unit layers is not really a NN but anyway let's see how it does

run_backtest(X, Y, create_model(3, 1, 0.0), 
             coef_dict='all', 
             initial_train_months=FIRST_TRAIN_MONTHS)

# slightly worse OOS MSE than linear regression (45.2) but much worse Sharpe

................................................................................
10:02:02 Still training 80 of 577
................................................................................
10:05:59 Still training 160 of 577
................................................................................
10:10:25 Still training 240 of 577
................................................................................
10:16:10 Still training 320 of 577
................................................................................
10:22:12 Still training 400 of 577
................................................................................
10:28:23 Still training 480 of 577
................................................................................
10:34:42 Still training 560 of 577
................
MSE across all predictions: 41.2533
Variance: 39.5535
R-squared: -0.0430


In [25]:
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)


Food.lead: long 107 times, short 70 times, total 177 times
Beer.lead: long 128 times, short 97 times, total 225 times
Smoke.lead: long 193 times, short 83 times, total 276 times
Games.lead: long 199 times, short 154 times, total 353 times
Books.lead: long 126 times, short 116 times, total 242 times
Hshld.lead: long 77 times, short 99 times, total 176 times
Clths.lead: long 162 times, short 155 times, total 317 times
Hlth.lead: long 107 times, short 78 times, total 185 times
Chems.lead: long 50 times, short 128 times, total 178 times
Txtls.lead: long 146 times, short 116 times, total 262 times
Cnstr.lead: long 85 times, short 108 times, total 193 times
Steel.lead: long 69 times, short 189 times, total 258 times
FabPr.lead: long 55 times, short 108 times, total 163 times
ElcEq.lead: long 122 times, short 85 times, total 207 times
Autos.lead: long 77 times, short 148 times, total 225 times
Carry.lead: long 114 times, short 136 times, total 250 times
Mines.lead: long 149 times, short 117 t

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,0.0126664
yearly_vol,0.0448401
yearly_sharpe,0.335211
max_drawdown,-0.129298
sortino,0.135873


In [71]:
# try keras instead of sklearn MLPRegressor
# wrap their model in a class 
# we are using multioutput to speed things up 
# fit takes a list of response ys, predict returns a list of y_predict arrays
INPUT_DIM = X.shape[1]
print(INPUT_DIM)
OUTPUT_DIM = len(responses) # 30
BATCH_SIZE = 32
EPOCHS=80

class KerasBacktestModel(object):

    def __init__(self, 
                 n_hidden_layers = 2,
                 hidden_layer_size = 32,
                 reg_penalty = 0.0001,
                 dropout = 0.333,
                 verbose=True):
        main_input = Input(shape=(INPUT_DIM,),
                           dtype='float32', 
                           name='main_input')
        lastlayer=main_input
        
        for i in range(n_hidden_layers):
            if verbose:
                print("layer %d size %d, reg_penalty %.8f, dropout %.3f" % (i + 1, 
                                                                            hidden_layer_size, 
                                                                            reg_penalty, 
                                                                            dropout))
            lastlayer = Dense(units = hidden_layer_size, 
                              activation = 'relu',
                              kernel_initializer = keras.initializers.glorot_uniform(),
                              kernel_regularizer=keras.regularizers.l1(reg_penalty),
                              name = "Dense%02d" % i)(lastlayer)
            
            if dropout:
                lastlayer = Dropout(dropout, name = "Dropout%02d" % i)(lastlayer)
                
        outputs = []
        for i in range(OUTPUT_DIM):
            # OUTPUT_DIM outputs
            output = Dense(1,
                           activation='linear',
                           name='output%02d' % i)(lastlayer)
            outputs.append(output)
            
        self.model = Model(inputs=[main_input], outputs=outputs)
        if verbose:
            print(self.model.summary())
            
        self.model.compile(loss="mse", optimizer="rmsprop", loss_weights=[1.]*OUTPUT_DIM)
        
    def fit(self, X, Y, epochs=EPOCHS):
        # convert Y to list of ys
        Y_list = [Y[:,i] for i in range(OUTPUT_DIM)]
        return self.model.fit(X,
                              Y_list,
                              batch_size=BATCH_SIZE,
                              epochs=epochs,
                              verbose=False)
    
    def predict(self, X):
        # convert list of ys to Y array
        y_list = self.model.predict(X)
        npreds=len(y_list[0])
        Y_xval_pred = [y.reshape(npreds) for y in y_list]
        Y_xval_pred = np.array(Y_xval_pred).transpose()    
        return Y_xval_pred
    
    def save(self, modelname):
        self.model.save("%s.h5" % modelname)
        self.model.save_weights("%s_weights.h5" % modelname)


103


In [87]:
def create_keras_model(n_hidden_layers, layer_size, reg_penalty, dropout, verbose=False):
    def create_func():
        return KerasBacktestModel(n_hidden_layers = n_hidden_layers,
                                  hidden_layer_size = layer_size,
                                  reg_penalty = reg_penalty,
                                  dropout = dropout,
                                  verbose=verbose)
    return create_func


In [50]:
def fit_predict_keras(X, Y, model, epochs=EPOCHS, npredict=1, verbose=False):
    """simpler fit_predict, no coef_dict, fits all at once, specifies epochs 
    for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final month
    if npredict=26, fit using n-26 rows, return prediction using X for final 26 months"""
    
    nrows = X.shape[0]
    if verbose:
        print("Fit on %d rows 0 to %d" % (nrows-npredict, nrows-npredict-1))
        print("Predict on %d rows %d to %d" % (npredict, nrows-npredict, nrows-1))
        
    # keep last rows to predict against
    X_predict = X[-npredict:]
    X_predict = X_predict.reshape(npredict,X.shape[1])
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]
    
    fit = model.fit(
        X_fit,
        Y_fit,
        epochs=epochs
    )
    
    return model.predict(X_predict)

print("%s Start fit" % (time.strftime("%H:%M:%S")))
keras_create_model=create_keras_model(1, 4, 0.01, 0.25)
keras_model=keras_create_model(verbose=True)
predictions = fit_predict_keras(X, Y, keras_model, npredict=3, epochs=3)
print("%s End fit" % (time.strftime("%H:%M:%S")))

predictions

11:39:28 Start fit
layer 1 size 4, reg_penalty 0.01000000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
__________________________________________________________________________________________________
Dense00 (Dense)                 (None, 4)            416         main_input[0][0]                 
__________________________________________________________________________________________________
Dropout00 (Dropout)             (None, 4)            0           Dense00[0][0]                    
__________________________________________________________________________________________________
output00 (Dense)                (None, 1)            5           Dropout00[0][0]                  
____________________________________

array([[ 3.0632823 , -0.14469436,  3.5400436 , -1.1777619 ,  0.7302627 ,
         0.20733154,  0.48030537, -2.8965492 ,  1.1980777 , -1.3838048 ,
         0.74006546,  0.9542984 ,  0.53694916,  3.2810624 , -0.40520313,
         3.1337516 , -1.0691235 , -0.15243937,  0.67836004,  0.23011701,
        -1.0680192 ,  0.54767406, -1.6263189 ,  0.79566485, -1.1569383 ,
         2.5313826 , -1.0813317 , -1.924275  , -3.1329238 , -0.55992997],
       [ 1.2874429 ,  0.7980152 ,  1.4525441 , -0.5404324 , -0.2224832 ,
         0.3672167 ,  0.11850676, -1.2202029 ,  0.7758051 , -1.3226769 ,
         1.0245316 , -0.31393254, -0.218428  ,  1.347803  ,  0.32448357,
         1.2431568 , -0.37896353, -0.7740058 , -0.3068104 ,  0.84744513,
        -0.2652901 ,  1.0738955 , -0.7275055 , -0.09550111, -1.3010827 ,
         1.031365  , -0.3155843 , -0.96316755, -1.1537324 ,  0.13930702],
       [-0.6188089 , -0.5800356 , -0.6243864 ,  0.99458295,  0.62529397,
        -0.5757832 , -0.8252713 ,  0.07836729, -0

In [58]:
EPOCHS=500
def walkforward_xval_keras (X, Y, create_model, n_splits=5, epochs=EPOCHS):
    ### no coef_dict, fit all at once

    # generate k-folds
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    model = create_model()
    
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        fit = model.fit(X_fit, Y_fit, epochs=epochs)
        # evaluate ... run prediction, calc MSE by industry, and average
        Y_xval_pred = model.predict(X_xval)
        
        mse_list = [mean_squared_error(Y_xval[:,i], Y_xval_pred[:,i]) for i in range(OUTPUT_DIM)]
        xval_score = np.mean(np.array(mse_list))            

        print ("\n%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("%s Avg Xval loss %f" % (time.strftime("%H:%M:%S"), avg_loss))
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [59]:

walkforward_xval_keras(X, Y, 
                       create_keras_model(n_hidden_layers=3,
                                          layer_size=2,
                                          reg_penalty=0.001,
                                          dropout=0.25),
                      epochs=5)

11:44:45 Generate splits [140, 280, 419, 558, 697]
11:44:45 Starting training
layer 1 size 2, reg_penalty 0.00100000, dropout 0.250
layer 2 size 2, reg_penalty 0.00100000, dropout 0.250
layer 3 size 2, reg_penalty 0.00100000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
__________________________________________________________________________________________________
Dense00 (Dense)                 (None, 2)            208         main_input[0][0]                 
__________________________________________________________________________________________________
Dropout00 (Dropout)             (None, 2)            0           Dense00[0][0]                    
___________________________________________________________________


11:45:04 Xval MSE 42.516294
11:45:04 Avg Xval loss 40.864027
--------------------------------------------------------------------------------


(40.86402743810781, <__main__.KerasBacktestModel at 0x7f3593b1c210>)

In [65]:
# run an experiment with walk-forward cross-validation

EPOCHS = 50

def run_experiment(X, Y, 
                   n_hidden_layers, 
                   layer_size, 
                   reg_penalty,
                   dropout,
                   minmaxscale=False, 
                   standardscale=False,
                   epochs=EPOCHS):
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")

    return walkforward_xval_keras(Xscale, Yscale,
                                  create_keras_model(n_hidden_layers=n_hidden_layers,
                                                     layer_size=layer_size,
                                                     reg_penalty=reg_penalty,
                                                     dropout=dropout),
                                  epochs=epochs)


In [66]:
run_experiment(X, Y, 3, 4, .001, .25, epochs=40)

11:49:44 Generate splits [140, 280, 419, 558, 697]
11:49:44 Starting training
layer 1 size 4, reg_penalty 0.00100000, dropout 0.250
layer 2 size 4, reg_penalty 0.00100000, dropout 0.250
layer 3 size 4, reg_penalty 0.00100000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
__________________________________________________________________________________________________
Dense00 (Dense)                 (None, 4)            416         main_input[0][0]                 
__________________________________________________________________________________________________
Dropout00 (Dropout)             (None, 4)            0           Dense00[0][0]                    
___________________________________________________________________


11:51:42 Xval MSE 41.112351
11:51:42 Avg Xval loss 38.235783
--------------------------------------------------------------------------------


(38.235782872409786, <__main__.KerasBacktestModel at 0x7f358a73b950>)

In [72]:
# improvement in MSE vs. LinearRegression (40ish vs. 54)
# run a lot of experiments in big xval loop to pick best hyperparameters

MODELPREFIX = "FFNN"
EPOCHS=200

n_hiddens = [1, 2, 3]
layer_sizes = [2, 4, 8]
reg_penalties = [0.0, 0.0001, 0.001, 0.01]
dropouts = [0.25]

hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties, dropouts))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty, dropout = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty, dropout)
    score, model = run_experiment(X, Y,
                                  n_hidden_layers = n_hidden_layers,
                                  layer_size = layer_size,
                                  reg_penalty = reg_penalty,
                                  dropout = dropout,
                                  epochs=EPOCHS
                                 )
    experiments[key] = score 
    modelname = "%s_%.6f_%d_%d_%.6f_%.3f" % (MODELPREFIX, score, n_hidden_layers, layer_size, reg_penalty, dropout)
    print("%s Saving %s.h5" % (time.strftime("%H:%M:%S"), modelname))
    model.save(modelname)


12:35:07 Running 36 experiments
12:35:07 Running experiment 1 of 36
12:35:07 Generate splits [140, 280, 419, 558, 697]
12:35:07 Starting training
layer 1 size 2, reg_penalty 0.00000000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
__________________________________________________________________________________________________
Dense00 (Dense)                 (None, 2)            208         main_input[0][0]                 
__________________________________________________________________________________________________
Dropout00 (Dropout)             (None, 2)            0           Dense00[0][0]                    
__________________________________________________________________________________________________
output00

Training indexes 0 to 279
Cross-validating indexes 280 to 418

12:46:34 Xval MSE 33.232254
Training indexes 0 to 418
Cross-validating indexes 419 to 557

12:49:40 Xval MSE 50.382995
Training indexes 0 to 557
Cross-validating indexes 558 to 696

12:53:41 Xval MSE 55.605630
12:53:41 Avg Xval loss 46.406960
--------------------------------------------------------------------------------
12:53:41 Saving FFNN_46.406960_1_2_0.000100_0.250.h5
12:53:51 Running experiment 3 of 36
12:53:51 Generate splits [140, 280, 419, 558, 697]
12:53:51 Starting training
layer 1 size 2, reg_penalty 0.00100000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
_______________________________________________________________________________________________

Training indexes 0 to 279
Cross-validating indexes 280 to 418

13:05:25 Xval MSE 32.391527
Training indexes 0 to 418
Cross-validating indexes 419 to 557

13:08:31 Xval MSE 58.442952
Training indexes 0 to 557
Cross-validating indexes 558 to 696

13:12:31 Xval MSE 43.553308
13:12:31 Avg Xval loss 44.795929
--------------------------------------------------------------------------------
13:12:31 Saving FFNN_44.795929_1_2_0.010000_0.250.h5
13:12:43 Running experiment 5 of 36
13:12:43 Generate splits [140, 280, 419, 558, 697]
13:12:43 Starting training
layer 1 size 4, reg_penalty 0.00000000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
_______________________________________________________________________________________________

Training indexes 0 to 279
Cross-validating indexes 280 to 418

13:24:19 Xval MSE 34.863183
Training indexes 0 to 418
Cross-validating indexes 419 to 557

13:27:25 Xval MSE 66.336679
Training indexes 0 to 557
Cross-validating indexes 558 to 696

13:31:24 Xval MSE 52.730681
13:31:24 Avg Xval loss 51.310181
--------------------------------------------------------------------------------
13:31:24 Saving FFNN_51.310181_1_4_0.000100_0.250.h5
13:31:38 Running experiment 7 of 36
13:31:38 Generate splits [140, 280, 419, 558, 697]
13:31:38 Starting training
layer 1 size 4, reg_penalty 0.00100000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
_______________________________________________________________________________________________

Training indexes 0 to 279
Cross-validating indexes 280 to 418

13:43:21 Xval MSE 32.974242
Training indexes 0 to 418
Cross-validating indexes 419 to 557

13:46:29 Xval MSE 75.238836
Training indexes 0 to 557
Cross-validating indexes 558 to 696

13:50:29 Xval MSE 51.469034
13:50:29 Avg Xval loss 53.227371
--------------------------------------------------------------------------------
13:50:29 Saving FFNN_53.227371_1_4_0.010000_0.250.h5
13:50:44 Running experiment 9 of 36
13:50:44 Generate splits [140, 280, 419, 558, 697]
13:50:44 Starting training
layer 1 size 8, reg_penalty 0.00000000, dropout 0.250
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
_______________________________________________________________________________________________

Training indexes 0 to 279
Cross-validating indexes 280 to 418


KeyboardInterrupt: 

In [ ]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]

lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "dropout",
                                            "loss"])
lossframe.sort_values(['loss'])
# better than LinearRegression or MLPRegressor

In [ ]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


In [ ]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())

In [ ]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())

In [ ]:
plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" units", y_suffix=" layers")


In [ ]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix="p", y_suffix=" layers")

In [ ]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" units", y_suffix="p")


In [91]:
EPOCHS=500

nrows = X.shape[0]
startindex = 1000

def run_backtest_keras(X, Y, arg_dict, startindex=0, epochs=EPOCHS, step=1, minmaxscale=False, standardscale=False):
    """create keras model; add step, to iteratively train, predict 12 months, train up to next 12 months """
    global P
    global R 
    
    print("%s Starting backtest" % (time.strftime("%H:%M:%S")))
    P = np.zeros((Y.shape[0],OUTPUT_DIM))
    
    count = 0
    nrows = X.shape[0]

    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")
     
    model = create_keras_model(n_hidden_layers=arg_dict["n_hidden_layers"],
                               layer_size=arg_dict["hidden_layer_size"],
                               reg_penalty=arg_dict["reg_penalty"],
                               dropout=dropout,
                               verbose=False)()
        
    for train_index in range(startindex, nrows, step):
        if train_index + step >= nrows:
            train_index = nrows-step
            
        fp_index = train_index + step # eg 1000 + 26 = 1026

        # fit on e.g. 0:999, predict 1000-1025
        predictions = fit_predict_keras(Xscale[:fp_index, :], 
                                        Yscale[:fp_index], 
                                        model,
                                        epochs=epochs,
                                        npredict=step)
        # store in 1000:1025 - lining up with future Xs
        for i in range(step):
            P[train_index + i]= predictions[i]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, nrows-startindex))
            sys.stdout.flush()

    msetemp = (P[startindex:]-Yscale[startindex:])**2
    #remove nans
    msetemp = msetemp[~np.isnan(msetemp)]
    mse = np.mean(msetemp)
    
    print("MSE across all predictions: %.4f" % mse)


In [92]:
def gen_returns(Y, P, first_pred_month, start_date='01/01/1970', freq='M', verbose=False):
    #TODO: more general version
    #take an indicator (P), a function to generate portfolio based on indicator, universe returns (Y), return portfolio returns

    global R
    R = np.zeros(P.shape[0])
    nrows, ncols = P.shape
    numstocks = 6 # top quintile (and bottom)

    indcount = [0 for response in responses]
    longcount = [0 for response in responses]
    shortcount = [0 for response in responses]
        
    for month_index in range(first_pred_month, nrows):
        # get indexes of sorted smallest to largest
        # leftmost 6
        # ignore nan
        short_sort_array = [999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(short_sort_array)
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_sort_array = [-999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(long_sort_array)
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        return_month = month_index + 1
        if verbose:
            print("Longs for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in long_indexes])))
            print("Shorts for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in short_indexes])))
            
        if return_month < nrows: # last row has a prediction for following month but no following month
            R[return_month] = np.mean(X[return_month, long_indexes])/2 - np.mean(X[return_month, short_indexes])/2
            # count occurrences of each industry
            for i in short_indexes:
                indcount[i]+=1
                shortcount[i]+=1
            for i in long_indexes:
                indcount[i]+=1
                longcount[i]+=1

    for response in responses:
        i = response_reverse_dict[response]
        print("%s: long %d times, short %d times, total %d times" % (response, longcount[i], shortcount[i], indcount[i]))
        
    results = R[first_pred_month:]

    index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [93]:
START=121
EPOCHS=500
STEP=5
arg_dict = {"n_hidden_layers" : 3,
            "hidden_layer_size" : 4,
            "reg_penalty" : 0.001,
            "dropout": 0.25,
            'verbose' : False
           }
     
#model = build_model(**arg_dict)
run_backtest_keras(X, Y, arg_dict, startindex=START, step=STEP, epochs=EPOCHS)


14:15:04 Starting backtest
................................................................................
15:04:51 Still training 80 of 576
................................................................................
16:17:14 Still training 160 of 576
................................................................................
17:52:25 Still training 240 of 576
................................................................................
19:50:25 Still training 320 of 576
................................................................................
22:11:12 Still training 400 of 576
................................................................................
00:54:26 Still training 480 of 576
................................................................................
04:00:19 Still training 560 of 576
....................MSE across all predictions: 51.6907


In [95]:
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)


Food.lead: long 192 times, short 67 times, total 259 times
Beer.lead: long 52 times, short 63 times, total 115 times
Smoke.lead: long 234 times, short 88 times, total 322 times
Games.lead: long 328 times, short 196 times, total 524 times
Books.lead: long 60 times, short 8 times, total 68 times
Hshld.lead: long 46 times, short 31 times, total 77 times
Clths.lead: long 92 times, short 35 times, total 127 times
Hlth.lead: long 84 times, short 45 times, total 129 times
Chems.lead: long 0 times, short 254 times, total 254 times
Txtls.lead: long 79 times, short 20 times, total 99 times
Cnstr.lead: long 0 times, short 115 times, total 115 times
Steel.lead: long 0 times, short 328 times, total 328 times
FabPr.lead: long 1 times, short 19 times, total 20 times
ElcEq.lead: long 122 times, short 4 times, total 126 times
Autos.lead: long 0 times, short 97 times, total 97 times
Carry.lead: long 226 times, short 70 times, total 296 times
Mines.lead: long 119 times, short 129 times, total 248 times
C

,Value
start,1970-01-31 00:00:00
end,2017-12-31 00:00:00
cagr,-0.0127782
yearly_vol,0.0625832
yearly_sharpe,-0.176748
max_drawdown,-0.546734
sortino,-0.0815138
